In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.initializers import glorot_uniform
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import cv2
import pydot

In [ ]:
#load_dataset function to load the data and resize the images to 50x50
def load_dataset(directory):
  images = []
  labels = []
  for idx, label in enumerate(uniq_labels):
    for file in os.listdir(directory + '/'+label):
      filepath = directory +'/'+ label + "/" + file
      img = cv2.resize(cv2.imread(filepath),(50,50))
      images.append(img)
      labels.append(idx)
  images = np.asarray(images)
  labels = np.asarray(labels)
  return images, labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#loading_dataset into X_pre and Y_pre
data_dir = '/content/drive/My Drive/Gesture Image Data'
uniq_labels = sorted(os.listdir(data_dir))
X_pre, Y_pre = load_dataset(data_dir)
print(X_pre.shape, Y_pre.shape)

(55501, 50, 50, 3) (55501,)


In [ ]:
#spliting dataset into 80% train, 10% validation and 10% test data
X_train, X_test, Y_train, Y_test = train_test_split(X_pre, Y_pre, test_size = 0.8)
X_test, X_eval, Y_test, Y_eval = train_test_split(X_test, Y_test, test_size = 0.5)

classes = np.array(uniq_labels)
classes = np.unique(classes)

In [ ]:
#print shapes and show examples for each set
print("Train images shape",X_train.shape, Y_train.shape)
print("Test images shape",X_test.shape, Y_test.shape)
print("Evaluate image shape",X_eval.shape, Y_eval.shape)
print("Printing the labels",uniq_labels, len(uniq_labels))

Train images shape (11100, 50, 50, 3) (11100,)
Test images shape (22200, 50, 50, 3) (22200,)
Evaluate image shape (22201, 50, 50, 3) (22201,)
Printing the labels ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_'] 37


In [ ]:
# converting Y_tes and Y_train to One hot vectors using to_categorical
# example of one hot => '1' is represented as [0. 1. 0. . . . . 0.]
Y_train = to_categorical(Y_train, num_classes=len(uniq_labels))
Y_test = to_categorical(Y_test, num_classes=len(uniq_labels))
Y_eval = to_categorical(Y_eval, num_classes=len(uniq_labels))
X_train = X_train / 255.
X_test = X_test/ 255.
X_eval = X_eval/ 255.

In [ ]:
def identity_block(X, filters, stage, block):
    """

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (3, 3), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def convolutional_block(X, filters, stage, block):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    # Retrieve Filters
    F1, F2 = filters

    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    # First component of main path
    X = Conv2D(F1, (3, 3), strides = (2, 2), padding = 'same', name = conv_name_base + '2a', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (3, 3), strides = (1, 1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform())(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)

    ##### SHORTCUT PATH ####
    X_shortcut = Conv2D(F2, (3, 3), strides = (2, 2), padding = 'same', name = conv_name_base + '1', kernel_initializer = glorot_uniform())(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [ ]:
def ResNet18(input_shape = (28, 28, 1), classes = 24):
    """
    Implementation of the popular ResNet18

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X = X_input = Input(input_shape)


    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    #X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, [64, 64], stage=2, block='a')
    X = identity_block(X, [64, 64], stage=2, block='b')

    # Stage 3
    X = convolutional_block(X, [128, 128], stage=3, block='a')
    X = identity_block(X, [128, 128], stage=3, block='b')

    # Stage 4
    X = convolutional_block(X, [256, 256], stage=4, block='a')
    X = identity_block(X, [256, 256], stage=4, block='b')

    # Stage 5
    X = convolutional_block(X, [512, 512], stage=5, block='a')
    X = identity_block(X, [512, 512], stage=5, block='b')

    # AVGPOOL
    # X = AveragePooling2D(pool_size=(2,2), name='avg_pool')(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)

    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet18')

    return model

In [ ]:
# train the neural network
inputs = tf.keras.layers.Input(shape=(50, 50, 3), name="input_layer")  # Set correct name
model = ResNet18(input_shape = (50, 50, 3), classes = len(uniq_labels))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

# test the neural network
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

Epoch 1/2
347/347 ━━━━━━━━━━━━━━━━━━━━ 1128s 3s/step - accuracy: 0.5827 - loss: 2.1652
Epoch 2/2
347/347 ━━━━━━━━━━━━━━━━━━━━ 1102s 3s/step - accuracy: 0.9863 - loss: 0.0486
694/694 ━━━━━━━━━━━━━━━━━━━━ 142s 203ms/step - accuracy: 0.6455 - loss: 6.3009
Loss = 6.3838419914245605
Test Accuracy = 0.6399099230766296


In [ ]:
pip install openvino==2023.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 6.6 MB/s eta 0:00:00


In [ ]:
import openvino as ov

In [ ]:
X_train[0]

array([[[0.23137255, 0.36862745, 0.5254902 ],
        [0.23137255, 0.36862745, 0.5254902 ],
        [0.22745098, 0.37254902, 0.52156863],
        ...,
        [0.39215686, 0.48235294, 0.56862745],
        [0.35686275, 0.4745098 , 0.54901961],
        [0.35686275, 0.48235294, 0.55686275]],

       [[0.25098039, 0.38823529, 0.54509804],
        [0.24313725, 0.38823529, 0.5372549 ],
        [0.24313725, 0.38431373, 0.53333333],
        ...,
        [0.38823529, 0.48235294, 0.56862745],
        [0.36470588, 0.47843137, 0.56078431],
        [0.36078431, 0.48627451, 0.56078431]],

       [[0.23921569, 0.38431373, 0.53333333],
        [0.24313725, 0.38823529, 0.5372549 ],
        [0.24705882, 0.38823529, 0.5372549 ],
        ...,
        [0.36470588, 0.47058824, 0.55294118],
        [0.35686275, 0.47843137, 0.56078431],
        [0.35686275, 0.48627451, 0.56470588]],

       ...,

       [[0.29019608, 0.34117647, 0.46666667],
        [0.2627451 , 0.34117647, 0.4627451 ],
        [0.25098039, 0

In [ ]:
# Create tensor example with X_train image
input_tensor = tf.convert_to_tensor(X_train)

In [ ]:
# Ensure X_train is float32 and converted to NumPy
X_train = tf.cast(X_train, tf.float32).numpy()

# Debug: Print the actual input layer name
print("Model Input Name:", model.input.name)  # Check the real input name

# Extract the correct input layer name (remove ":0" if present)
input_layer_name = model.input.name.split(":")[0]

# Debug: Ensure input layer name matches OpenVINO expectation
print("Using Input Name:", input_layer_name)

# Ensure the example input uses the correct key and is a single sample
example_input = {input_layer_name: X_train[:1]}  # Provide a batch of 1

# Convert the model to OpenVINO format
ov_model = ov.convert_model(model, example_input=example_input)

Model Input Name: keras_tensor_71
Using Input Name: keras_tensor_71


Exception: Could not trace the TF model with the following error: Binding inputs to tf.function failed due to `Attempt to convert a value ({'keras_tensor_71': array([[[[0.23137255, 0.36862746, 0.5254902 ],
         [0.23137255, 0.36862746, 0.5254902 ],
         [0.22745098, 0.37254903, 0.52156866],
         ...,
         [0.39215687, 0.48235294, 0.5686275 ],
         [0.35686275, 0.4745098 , 0.54901963],
         [0.35686275, 0.48235294, 0.5568628 ]],

        [[0.2509804 , 0.3882353 , 0.54509807],
         [0.24313726, 0.3882353 , 0.5372549 ],
         [0.24313726, 0.38431373, 0.53333336],
         ...,
         [0.3882353 , 0.48235294, 0.5686275 ],
         [0.3647059 , 0.47843137, 0.56078434],
         [0.36078432, 0.4862745 , 0.56078434]],

        [[0.23921569, 0.38431373, 0.53333336],
         [0.24313726, 0.3882353 , 0.5372549 ],
         [0.24705882, 0.3882353 , 0.5372549 ],
         ...,
         [0.3647059 , 0.47058824, 0.5529412 ],
         [0.35686275, 0.47843137, 0.5607843

In [ ]:
#Export the model to ONNX format
!pip install tf2onnx
import tf2onnx
import onnx

In [ ]:
# Convert the model to ONNX
model.export("ResNet18.onnx",format="onnx")

Saved artifact at 'ResNet18.onnx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 50, 50, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 37), dtype=tf.float32, name=None)
Captures:
  133133626415952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626418640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626418448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626409808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626417296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626413648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626411728: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626416720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626417104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626416336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133133626416528: T